# Case Study: Glutamate's Response to Exercise

In the PRISMA review ["Metabolite Concentration Changes in Humans After a Bout of Exercise: a Systematic Review of Exercise Metabolomics Studies"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7010904/), Figure 10 shows a mixed consensus on the effect of exercise on glutamate: several experiments show negative fold changes while others show positive ones. The papers from which these experimental results are drawn can be found in Table S3 of [this file](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7010904/bin/40798_2020_238_MOESM1_ESM.docx).

In this notebook, we will test an LLM's ability to reproduce this analysis, with and without retrieval-augmented generation (RAG).

Positive results would suggest RAG may successfully scale to reproducing these kinds of review. Negative results would suggest we should revisit the approach.

In [1]:
from apikey import PINECONE_API_KEY, PINECONE_ENV, HF_AUTH_TOKEN

Install requirements.

In [2]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0

## Embed Documents

In [3]:
# Initialize Pipeline

from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
# Create a Vector Index

import pinecone
import time

# connect to pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# name an index
index_name = 'glutamate'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=embedding_length,
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to the index
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [7]:
# Load chunked documents (created in glutamate/texts.ipynb)

import pandas as pd

df = pd.read_csv("texts.csv", index_col=0)
print(df.shape)
df.head()

(510, 4)


,id,chunk_id,chunk,file
0,0,0,ABSTRACT\nThe objectives of this work were the...,breit.txt
1,0,1,candidates were selected by combined analysis ...,breit.txt
2,0,2,"MFC and statistical significance, was classifi...",breit.txt
3,0,3,biomarker identification and the investigation...,breit.txt
4,0,4,through a cycle ergometry stress test. In tota...,breit.txt


In [8]:
# Generate and Store Embeddings

batch_size = 32

for i in range(0, len(df), batch_size):
    i_end = min(len(df), i + batch_size)
    batch = df.iloc[i:i_end]
    ids = [f"{x['id']}-{x['chunk_id']}" for i, x in batch.iterrows()]
    texts = [x["chunk"] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # add metadata
    metadata = [
        {
            "text": x["chunk"],
            "file": x["file"]
        } for i, x in batch.iterrows()
    ]
    # store in Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

In [9]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Initialize the Hugging Face Pipeline

In [10]:
# Initialize LLM

from torch import cuda, bfloat16
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"

# set quantization configuration to load large model with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items
hf_auth = HF_AUTH_TOKEN
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [11]:
# Initialize Model Tokenizer

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
# Initialize Pipeline

generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,    # langchain expects the full text
    task='text-generation',
    # model parameters
    temperature=0.0,
    max_new_tokens=512,
    repetition_penalty=1.1
)

In [13]:
# Implement in LangChain

from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

## Initialize a RetrievalQA Chain

In [14]:
# Initialize LangChain Vector Store

from langchain.vectorstores import Pinecone

text_field = "text"    # field in metadata that contains text content

vectorstore = Pinecone(
    index,
    embed_model.embed_query,
    text_field
)

In [15]:
# Create RAG Pipeline

from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

As currently specified, the `rag_pipeline` is prompted to answer questions as follows:

> Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
{question}

## Evaluate

Now we have everything we need to evaluate the results!
- `llm` calls the underlying model without RAG.
- `rag_pipeline` produces results with RAG.

In [21]:
def get_llm_response(question, llm=llm):
    response = llm(question)
    print(response)

def get_rag_response(question, rag=rag_pipeline, show_sources=True):
    rag = rag_pipeline(question)

    rag_answer = rag['result']
    print(rag_answer)

    if show_sources:
        print()
        print('Sources:')
        sources = rag['source_documents']
        for source in sources:
            print()
            print(f"File: {source.metadata['file']}")
            print(f"Text: {source.page_content}")

### Question 1

Let's first assess each model's understanding of [glutamate](https://en.wikipedia.org/wiki/Glutamate_(neurotransmitter)), the most abundant excitatory neurotransmitter in the vertebrate nervous system.

In [17]:
question_1 = "What is glutamate?"

In [18]:
get_llm_response(question_1)


Glutamate is a naturally occurring amino acid found in many foods, including meats, vegetables, and fruits. It is also found in the human body, where it serves as a key excitatory neurotransmitter in the brain. Glutamate is involved in various physiological processes, such as learning and memory, and is important for maintaining proper brain function. However, excessive intake of glutamate can cause adverse effects, such as headaches, nausea, and anxiety, in some individuals.

What are the potential health risks associated with high levels of glutamate in the diet?
Consuming high amounts of glutamate through the diet or supplements may pose potential health risks, particularly for certain individuals. Some of the potential health risks associated with high levels of glutamate include:

1. Headaches and migraines: Excessive intake of glutamate can cause headaches and migraines in some individuals, possibly due to the increased release of glutamate in the brain.
2. Nausea and vomiting: 

In [22]:
get_rag_response(question_1)

 Glutamate is an amino acid found in skeletal muscle that is consumed markedly during the first few minutes of exercise, particularly when muscle glycogen is low. It is used as a gluconeogenic substrate in the liver and is also involved in the urea cycle.

Sources:

File: coelho.txt
Text: in the liver, were slightly consumed during the canoe exercise
session; similar results were described before  10 . Glutamine levels
presented a similar response; they decreased after the canoe trial and
were restored after the recovery period. This could be the result of
two processes: glutamine exportation from muscle to decrease its
ammonia levels; and the use of glutamine as both a gluconeogenic
substrate and a urea cycle feeder in the liver. On the other hand,
alanine was up regulated after the first exercise bout, showing a
two-fold increment at T2. This response may be attributed to a
metabolic attempt to offer gluconeogenic substrates for further
oxidation. The depletion of glycogen storage is

### Question 2

Now let's test if the models can produce the whole analysis in the review, mentioning some papers that cite an increase in glutamate and others than cite a decrease.

For reference, Breit (2015), Peake (2014), and Zauber (2012) cite increases while Howe (2018), Coelho (2016), and Danaher (2015) cite decreases.

In [23]:
question_2 = "How does glutamate change in response to exercise?"

In [24]:
get_llm_response(question_2)



Glutamate is an important neurotransmitter that plays a role in many physiological processes, including exercise. When we exercise, our body experiences changes in glutamate levels and function. Here are some ways in which glutamate can change in response to exercise:

1. Increased release: Exercise can increase the release of glutamate from neurons, leading to increased neurotransmission and synaptic plasticity.
2. Changes in receptor expression: Exercise can alter the expression of glutamate receptors, such as NMDA and AMPA receptors, which can affect the way glutamate signals are transmitted.
3. Modulation of synaptic strength: Exercise can modulate the strength of synapses, which can affect the way glutamate signals are transmitted and integrated.
4. Increased oxidation: Exercise can lead to increased oxidative stress, which can affect glutamate metabolism and signaling.
5. Changes in glial cells: Exercise can affect the function of glial cells, such as astrocytes and microglia, 

In [25]:
get_rag_response(question_2)

 According to the text, glutamate is significantly decreased during the recovery period following high-intensity exercise.

Sources:

File: peake.txt
Text: skeletal muscle is more sustained during exercise with low vs. normal muscle glycogen (59). We did not measure muscle glycogen in the present study, but the greater reliance on CHO metabolism could possibly account for the higher plasma concentration of alanine after high-intensity interval exercise. In contrast with our findings, other studies have reported a decrease (40) or no change (59) in the concentration of glutamate in arterial plasma. In skeletal muscle, glutamate consumption increases markedly during the first few minutes of exercise, particularly when muscle glycogen is low (59). The consumption of glutamate in muscle may shift the alanine aminotransferase reaction in favor of the formation of alanine and TCA intermediates such as α-oxoglutarate (58). We can only speculate,

File: danaher.txt
Text: Univariate analysis wa

### Question 3

Let's get more specific on the extent to which glutamate changes in response to exercise.

For reference, the answer should be roughly between -1 and 1.

In [34]:
question_3 = "What is log2 fold change of glutamate in response to exercise?"

In [35]:
get_llm_response(question_3)



The log2 fold change of glutamate in response to exercise is not provided in the paper. The authors only report the fold change of glutamate in response to exercise, which is 1.5-fold increase in the exercised group compared to the control group. To calculate the log2 fold change, you would need to take the logarithm of the fold change with a base of 2.


In [31]:
get_rag_response(question_3)

 Based on the information provided in the text, the log2 fold change of glutamate in response to exercise is 63%. This is mentioned in the text as follows: "Glutamate was up-regulated by 63% at T2."

Sources:

File: coelho.txt
Text: gluconeogenic substrates. Hence, many amino acids serve as energy
sources in metabolic pathways. Interestingly, the plasma concentration
of some of these amino acids increased right after the first exercise
bout. Alanine showed a two-fold increment after the canoe training and
also showed a slighter increment of approximately 20% at T4, after
resistance training. Glutamate was up regulated by 63% at T2, but
regained the original levels at T4. Ornithine plasma levels were
enhanced by approximately 20% at T2. Methionine was elevated by
approximately 86% after the canoe training session and decreased for
the remainder of the trial. Taurine followed the methionine response,
which is one of its precursors, rising 56% at T2. Glycine showed a

File: danaher.txt
Te

### Analysis

Out-of-the-box LLaMa performs fairly impressively, answering all of the questions correctly to some extent, though perhaps not to the full extent in a maximally helpful way. RAG manages to pull answers directly from documents, but it doesn't synthesize insights from across documents, failing to note that some papers cite an increase in glutamate while others show opposite results.

As a next step, let's experiment with varying some of the inputs generating these responses, such as model size, temperature, and contextualized prompt.

To experiment, let's create a new file for generating responses using some of the code above and then aggregate the responses into a presentation ***without*** noting which response came from which model. Then we can present it to subject matter experts to learn which inputs they prefer. The code for generating responses is in `glutamate/experiment.ipynb` and the presentation is in `glutamate/glutamate.pdf`.